## REQUIREMENTS

In [10]:
import sys
sys.path.append("/content/drive/MyDrive/DL_projects/text_classification/src")

In [141]:
import sys 
import json
import torch
import nltk
import pickle
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import re
import gensim
import config
import helper
from sklearn.model_selection import train_test_split
from model import ClassifierModel
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
sw = stopwords.words('english')
from gensim.models import KeyedVectors
import model
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [161]:
config.local_base_dir = "/content/drive/MyDrive/DL_projects/text_classification/"
config.local_train_file = "dataset/train.tsv"
config.prep_train_test_vocabed = "prep_train_test_vocabed.pkl"
config.EMBED_SIZE = 32

In [152]:
data = pd.read_csv("/content/drive/MyDrive/DL_projects/text_classification/dataset/train.tsv",sep = '\t',header = None)

## PREPROCESSING

In [153]:
wordnet_lemmatizer = WordNetLemmatizer()
porter_stemmer  = PorterStemmer()

def preprocess_text(text):
    text = re.sub(r"^[A-Za-z]","",text)
    text = " ".join([x for x in text.split() if x not in sw])
    text = " ".join([wordnet_lemmatizer.lemmatize(x) for x in text.split() ])
    # text = " ".join([porter_stemmer.stem(x) for x in text.split()])
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

In [154]:
def data_length(dataframe):
    all_words = set()
    all_sentence = list()
    for sentence in dataframe["prep_data"]:
        all_sentence.append(sentence.lower().split())
        for word in sentence.lower().split():
            all_words.add(word)
    len_all_words = len(all_words)
    len_all_sentence = len(all_sentence)
    print(f"Total number of words : {len_all_words}")
    print(f"Total number of sentence : {len_all_sentence}")
    print()
    return all_words,all_sentence

In [156]:
if __name__ == "__main__":
    prep_data = data[3].apply(lambda x : preprocess_text(x)) 
    data["prep_data"] = prep_data
    all_words,all_sentence = data_length(data)
    
    samples_lbl = data[1]
    X_train, X_test, y_train, y_test = train_test_split(all_sentence,samples_lbl,test_size=0.2,shuffle=True)


Total number of words : 148823
Total number of sentence : 35000



In [162]:

pickle.dump({
    "X_train" : X_train,
    "X_test" : X_test,
    "y_train" : y_train.values.tolist(),
    "y_test" : y_test.values.tolist(),
},open(config.local_base_dir + "dataset/"+ config.prep_train_test_vocabed,'wb'))



In [163]:
config.local_base_dir + "dataset/"+ config.prep_train_test_vocabed

'/content/drive/MyDrive/DL_projects/text_classification/dataset/prep_train_test_vocabed.pkl'

In [10]:
split_data = pickle.load(open(config.local_base_dir + "dataset/"+config.prep_train_test_vocabed,'rb'))

# **Word to Vectors**

In [164]:
def build_word2vec(all_sentence,embedding_size):
    w2v_model = gensim.models.Word2Vec(sentences=all_sentence,min_count=1,vector_size= embedding_size)
    w2v_model.build_vocab(all_sentence)
    print("Length of samples : ",w2v_model.corpus_count)
    print("Length of vocab   : ",len(w2v_model.wv.key_to_index))
    print("Training and saving model...")
    w2v_model.train(all_sentence,total_examples=w2v_model.corpus_count,epochs=w2v_model.epochs)
    w2v_model.save(f"{config.local_base_dir}dataset/prep_word2vectors_{config.EMBED_SIZE}.model")
    wordvecs = KeyedVectors.load(f"{config.local_base_dir}dataset/prep_word2vectors_{config.EMBED_SIZE}.model")    
    all_words = list(wordvecs.wv.key_to_index.keys())
# wordvecs.wv.index_to_key = {v:k for k,v in wordvecs.wv.key_to_index.items()}
    word2index = {k:v+1 for k,v in wordvecs.wv.key_to_index.items()}
    index2word = {v:k for k,v in word2index.items()}
    
    matrix_vec = np.zeros((len(word2index)+1,config.EMBED_SIZE))
    for word,idx in word2index.items():
        vector_x = wordvecs.wv[word]
        matrix_vec[idx,:] = vector_x    
    pickle_data = {
    "word2index" : word2index,
    "index2word" : index2word,
    "embedding_vector" : matrix_vec
    }
    pickle.dump(pickle_data,open(config.local_base_dir+"dataset/prep_emb_vec.pkl",'wb'))
    print("Done")
    print(f'Model saved to {config.local_base_dir+"dataset/prep_emb_vec.pkl"}')
    

In [165]:
config.EMBED_SIZE = 8
build_word2vec(all_sentence,config.EMBED_SIZE)

Length of samples :  35000
Length of vocab   :  148823
Training and saving model...
Done
Model saved to /content/drive/MyDrive/DL_projects/text_classification/dataset/prep_emb_vec.pkl


In [166]:
pickle_data =pickle.load(open("emb_vec.pkl",'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'emb_vec.pkl'

In [ ]:
# word2index = pickle_data['word2index']
# index2word = pickle_data['index2word']
# word_vectors = pd.DataFrame(pickle_data['embedding_vector'])
# word_vectors.to_csv("embedding_matrix.tsv",sep ="\t",header = None,index = False)
# index2word[0] = "[PAD]"
# only_words = [index2word[x] for x in range(len(index2word))]
# word_vectors["metadata"] =only_words
# word_vectors.to_csv("meta_data.tsv",sep ="\t",index = False)

In [2]:
checkpoint = torch.load("/content/drive/MyDrive/DL_projects/text_classification/trained_models/seq2seq_hidden_32_embed_32_word2vec.pt")
checkpoint.keys()

dict_keys(['model_state_dict', 'params'])

In [43]:
params = checkpoint['params']
pickle_data =pickle.load(open(params["base_dir"] +params["emb_vec_file"],'rb'))
pickle_data.keys()

word2index = pickle_data['word2index']
index2word = pickle_data['index2word']

In [44]:
# n_vocab,hidden_size,out_hidden,embedding_dim,n_labels,max_seq

my_model = model.ClassifierModel(len(word2index),params["HIDDEN_SIZE"],params["OUT_DIM"],params["EMBED_SIZE"],n_labels = params["n_labels"],max_seq=params['max_seq_len'])

In [45]:
my_model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [46]:
import pandas as pd
emb_matrix = my_model.embedding.weight.detach().numpy()

In [68]:
emb = pd.DataFrame(emb_matrix)

In [69]:
emb.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-8.009912,-2.296135,1.751826,-2.643408,1.582709,-3.423228,1.673678,2.366343,0.920419,3.158559,2.454304,0.311159,3.599848,1.312220,-0.423543,1.432479
2,0.168141,-1.772859,-0.500348,1.046328,3.021698,-7.178311,3.537433,-0.229177,0.014891,-0.685747,-1.323345,-1.795694,-0.414231,4.162427,-3.456668,4.509147
3,-3.558702,-1.454928,2.358758,1.284768,-0.278380,-2.398882,3.422882,2.132612,1.681724,-4.255305,0.222466,-0.502357,-1.533517,0.967723,-0.499883,-1.353392
4,-2.494306,-2.278086,1.807872,0.244650,3.597616,-1.243710,0.698184,-0.634613,-0.723817,-0.340771,3.004715,1.082289,4.934502,0.853312,0.782405,-0.290913


In [49]:
emb.shape

(140808, 16)

In [51]:
len(index2word)

140807

In [70]:
emb.to_csv("prep_embedding_matrix.tsv",sep = '\t',header=None,index = False)

In [73]:
pd.read_csv("/content/drive/MyDrive/DL_projects/text_classification/src/interpret/prep_embedding_matrix.tsv",sep = '\t').head()

,0.0,0.0.1,0.0.2,0.0.3,0.0.4,0.0.5,0.0.6,0.0.7,0.0.8,0.0.9,0.0.10,0.0.11,0.0.12,0.0.13,0.0.14,0.0.15
0,-8.009912,-2.296135,1.751826,-2.643408,1.582709,-3.423228,1.673678,2.366344,0.920419,3.158559,2.454304,0.311159,3.599848,1.312220,-0.423543,1.432478
1,0.168141,-1.772859,-0.500348,1.046328,3.021698,-7.178311,3.537433,-0.229177,0.014891,-0.685747,-1.323345,-1.795694,-0.414231,4.162427,-3.456668,4.509147
2,-3.558702,-1.454928,2.358758,1.284768,-0.278380,-2.398882,3.422882,2.132612,1.681724,-4.255305,0.222466,-0.502357,-1.533516,0.967723,-0.499883,-1.353392
3,-2.494306,-2.278086,1.807872,0.244650,3.597616,-1.243710,0.698184,-0.634613,-0.723817,-0.340771,3.004715,1.082288,4.934502,0.853312,0.782405,-0.290913
4,-4.728490,-1.770580,1.477018,0.332946,2.624093,-2.911763,0.631502,-0.532540,0.631188,1.111279,0.748544,1.091287,5.950947,0.435730,1.386499,0.056322


In [77]:
index2word[0] = "[PAD]"

In [78]:
word_vectors = pd.read_csv("/content/drive/MyDrive/DL_projects/text_classification/src/interpret/prep_embedding_matrix.tsv",sep = '\t',header = None)
print(word_vectors.shape,len(index2word))
word_vectors.head()

(140808, 16) 140808


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-8.009912,-2.296135,1.751826,-2.643408,1.582709,-3.423228,1.673678,2.366344,0.920419,3.158559,2.454304,0.311159,3.599848,1.312220,-0.423543,1.432478
2,0.168141,-1.772859,-0.500348,1.046328,3.021698,-7.178311,3.537433,-0.229177,0.014891,-0.685747,-1.323345,-1.795694,-0.414231,4.162427,-3.456668,4.509147
3,-3.558702,-1.454928,2.358758,1.284768,-0.278380,-2.398882,3.422882,2.132612,1.681724,-4.255305,0.222466,-0.502357,-1.533516,0.967723,-0.499883,-1.353392
4,-2.494306,-2.278086,1.807872,0.244650,3.597616,-1.243710,0.698184,-0.634613,-0.723817,-0.340771,3.004715,1.082288,4.934502,0.853312,0.782405,-0.290913


In [79]:
only_words = [index2word[x] for x in range(len(index2word))]

In [87]:
word_vectors.shape,len(only_words),only_words[-1]

((140808, 17), 140808, 'burketsville')

In [88]:
word_vectors.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,metadata
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,[PAD]
1,-8.009912,-2.296135,1.751826,-2.643408,1.582709,-3.423228,1.673678,2.366344,0.920419,3.158559,2.454304,0.311159,3.599848,1.312220,-0.423543,1.432478,i
2,0.168141,-1.772859,-0.500348,1.046328,3.021698,-7.178311,3.537433,-0.229177,0.014891,-0.685747,-1.323345,-1.795694,-0.414231,4.162427,-3.456668,4.509147,br
3,-3.558702,-1.454928,2.358758,1.284768,-0.278380,-2.398882,3.422882,2.132612,1.681724,-4.255305,0.222466,-0.502357,-1.533516,0.967723,-0.499883,-1.353392,the
4,-2.494306,-2.278086,1.807872,0.244650,3.597616,-1.243710,0.698184,-0.634613,-0.723817,-0.340771,3.004715,1.082288,4.934502,0.853312,0.782405,-0.290913,film


In [82]:
word_vectors["metadata"] =only_words

In [83]:
word_vectors.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,metadata
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,[PAD]
1,-8.009912,-2.296135,1.751826,-2.643408,1.582709,-3.423228,1.673678,2.366344,0.920419,3.158559,2.454304,0.311159,3.599848,1.312220,-0.423543,1.432478,i
2,0.168141,-1.772859,-0.500348,1.046328,3.021698,-7.178311,3.537433,-0.229177,0.014891,-0.685747,-1.323345,-1.795694,-0.414231,4.162427,-3.456668,4.509147,br
3,-3.558702,-1.454928,2.358758,1.284768,-0.278380,-2.398882,3.422882,2.132612,1.681724,-4.255305,0.222466,-0.502357,-1.533516,0.967723,-0.499883,-1.353392,the
4,-2.494306,-2.278086,1.807872,0.244650,3.597616,-1.243710,0.698184,-0.634613,-0.723817,-0.340771,3.004715,1.082288,4.934502,0.853312,0.782405,-0.290913,film


In [89]:
word_vectors.to_csv("prep_meta_data.tsv",sep ="\t",index = False)

In [90]:
pd.read_csv("prep_meta_data.tsv",sep ="\t").head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,metadata
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,[PAD]
1,-8.009912,-2.296135,1.751826,-2.643408,1.582709,-3.423228,1.673678,2.366344,0.920419,3.158559,2.454304,0.311159,3.599848,1.312220,-0.423543,1.432478,i
2,0.168141,-1.772859,-0.500348,1.046328,3.021698,-7.178311,3.537433,-0.229177,0.014891,-0.685747,-1.323345,-1.795694,-0.414231,4.162427,-3.456668,4.509147,br
3,-3.558702,-1.454928,2.358758,1.284768,-0.278380,-2.398882,3.422882,2.132612,1.681724,-4.255305,0.222466,-0.502357,-1.533516,0.967723,-0.499883,-1.353392,the
4,-2.494306,-2.278086,1.807872,0.244650,3.597616,-1.243710,0.698184,-0.634613,-0.723817,-0.340771,3.004715,1.082288,4.934502,0.853312,0.782405,-0.290913,film


In [86]:
word_vectors

(140808, 17)

In [115]:
test_data = pd.read_csv("/content/drive/MyDrive/DL_projects/text_classification/dataset/train.tsv",sep = '\t',header = None)[[1,3]].iloc[:12,:]

In [130]:
for i in range(5):
    if test_data[1][i] == 1:
        print("HAM")
        print(test_data[3][i])
        print()

    else:
        print("SPAM")
        print(test_data[3][i])
        print()


# test_data[3]

HAM
I have done a lot of international travel, both on business and as a tourist. For both types I assure you the best advice is also the oldest: Always drink the wine of the country. In this movie the archangel Michael comes to Earth on business, wraps it up quickly and decides to hang around for a little touring. Boy! Does he "drink the wine of the country."<br /><br />Could man be drunk forever with liquor, love and fights <br /><br />He'd lief rise up of mornings and lief lie down of nights.<br /><br />These are things you can't do in Heaven so he enjoys them while he's here! Of course it turns out he had a couple of other jobs to tackle and, if he is less direct about them than he was about the first one, he is just as successful. The final scene is a little schmaltzy but it is also wonderful. Jean Stapleton gets to dance with John Travolta.

HAM
One of the most frightening game experiences ever that will make you keep the lights on next to your bed. Great storyline with a romanti